## Номер документа

Номер документа часто указывается после символа No или N и может состоять из цифр, букв и спецсимволов. Примеры номеров:

* 67-ФЗ,

* 324-01-ЗМО,

* 824н.

Номер документа необходимо извлечь из текста «как есть», не подвергая дополнительной обработке.
Поле number должно содержать извлеченную строку. Чтобы скомпенсировать влияние ошибок распознавания символов в исходных текстах, при сравнении предсказанного и настоящего номера проверяющей системой регистр букв не учитывается.


Для каждого типа документа свой извлекатор:

* указ - в конце документа на отдельной строке № 807, может быть и в начале № 256-уг

* постановление, распоряжение, приказ - в начале после даты и места (или перед местом) в конце строки № 688-п, №_70-пп_, 63-пр,  № 25/2-г

* закон, федеральный закон - в конце документа № 89-ОЗ

Особые случаи:

* № _ 643-уГ

* 20 ноября 2014 года

№ 596-5-ЗКО

в постановлении в конце документа

* 02 ноября 2015 г. л 23/557-П в постановлении в начале (будем выбирать в качестве номера окончание строки)

In [1]:
import re
import json

In [2]:
date_reg_exprs = [
    re.compile(r'от \d?\d [А-Яа-я]+ \d\d\d\d[^\n]+'),
    re.compile(r'\d\d\.\d\d\.\d\d\d\d[^\n]+'),
    re.compile(r'\n\d?\d [А-Яа-я]+ \d\d\d\d[^\n]+')
]

date_num_reg_exprs = [
    re.compile(r'от \d?\d [А-Яа-я]+ \d\d\d\d[^\n№]+№[^\n№]+'),
    re.compile(r'\d\d\.\d\d\.\d\d\d\d[^\n№]+№[^\n№]+'),
    re.compile(r'\n\d?\d [А-Яа-я]+ \d\d\d\d[^\n№]+№[^\n№]+')
]

number_reg_exprs = [
    re.compile(r' *№°?[_ ]+([\w\-/А-Яа-я]+)[_ ]*'),
    re.compile(r'\n№°?[_ ]+([\w\-/А-Яа-я]+)\n*'),
    re.compile(r'№°?([\w\-/А-Яа-я]+)')
]

In [3]:
s = "№ 876 \n № 897"
list(map(lambda x: x.group(1), list(number_reg_exprs[0].finditer(s))))

['876', '897']

In [4]:
number_reg_exprs[0].search(s).span()[0]

0

In [70]:
LAW_EXPR = re.compile('[ _]\d\d\d\d[ _]+г[а-я\.]* ?.?[\n]+№[ _]*([\w\-/А-Яа-я]+)')
SIMPLE_NUM = re.compile('№[_ ]*([\w\-/А-Яа-я]+)')

In [71]:
s = "№ 876 \n № 897"
SIMPLE_NUM.findall(s)

['876', '897']

In [72]:
from typing import List

class NumberExtractor: 
    def predict(self, test: List[str], types: List[str]) -> List[str]:
        pred_numbers = []
        for doc, doc_type in zip(test, types):
            doc = doc.lower()
            pred_number = ''
            if doc_type in ['федеральный закон', 'закон']:
                matches = LAW_EXPR.findall(doc)
                if matches:
                    pred_number = matches[-1]
                else:
                    matches = SIMPLE_NUM.findall(doc)
                    if matches:
                        pred_number = matches[-1]
            pred_numbers.append(pred_number)
        return pred_numbers

In [73]:
from eval_module import quality
from type_extractor import TypeExtractor
from sklearn.model_selection import train_test_split

train = []
names =[]
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append((f.read(), doc_dict['label']))
# train_data, test, _, names_test = train_test_split(train, names)

test = train
test_data = [x[0] for x in test]
test_labels = [x[1] for x in test]
true_types = [x[1]['type'] for x in test]

def predict(pred_numbers):
    results = []
    for pred_number in pred_numbers:
        prediction = {"type": "",
                      "date": "",
                      "number": pred_number,
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

# te = TypeExtractor()
# te.train(train_data)

ne = NumberExtractor()
predicted = ne.predict(test_data, true_types)
predicted = predict(predicted)
quality(predicted, test_labels)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0897468037,
 'type_f1_score': 0.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 0}

In [74]:
import numpy as np

true_types = np.array(true_types)
predicted = np.array(predicted)
test_labels = np.array(test_labels)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type}: {quality(predicted[ids], test_labels[ids])['number_accuracy']}")

федеральный закон: 1.0
постановление: 0.0
приказ: 0.0
распоряжение: 0.0
закон: 0.8341708543
указ: 0.0


In [75]:
doc_type = 'закон'
ids = np.where(true_types == doc_type)

y_test = [(x['number'], x['type']) for x in test_labels[ids]]
y_pred = [x['number'] for x in predicted[ids]]
names = np.array(names)
test_names = names[ids]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test[0].lower() != pred.lower():
        wrong_docs.append((test_names[i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f.read())
        print("=====")

right value = ('112-ОЗ', 'закон')
wrong value = 112-озо
РОССИЙСКАЯ ФЕДЕРАЦИЯ

ВЛАДИМИРСКАЯ ОБЛАСТЬ

ЗАКОН

О внесении изменения в статью 3 Закона Владимирской области
"О государственной поддержке и стимулировании деятельности
потребительской кооперации во Владимирской области"

Принят Законодательным Собранием области 24 сентября 2014 года

Статья 1

Внести в статью 3 Закона Владимирской области от 14 ноября 2007 года
№ 142-03 "О государственной поддержке и стимулировании деятельности
потребительской  кооперации - во Владимирской  области"  (Владимирские
ведомости, 2007, 21 ноября; 2011, $ июня; 2013, 16 ноября) изменение, изложив
абзац третий в следующей редакции:

"стимулирование организации закупок сельскохозяйственной продукции у
населения;".

Статья 2
Настоящий Закон вступает в силу после его официального опубликования.

  
    
 

х
3
©
И.о. Губернатор Отдеёо ВЕ *
. \ 5 \ делопроиз ®
Владимирской обласи Ь А.В. Конышев
ом /$)
\о °
\ЁЭ % *

Владимир
14 октября 2014 г.
№ 112-ОЗО Зак